In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import neighbors
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
from sklearn import metrics
from xgboost import XGBClassifier

In [3]:
from warnings import filterwarnings
filterwarnings('ignore')

In [4]:
df=pd.read_csv('./covid19_dataset.csv')
df=df.fillna(df.mean())
y=df["INTUBATION"]
z=df["INTENSIVE CARE"]
ID=df["ID"]
X=df.drop(["INTUBATION","INTENSIVE CARE","ID"],axis=1)

In [5]:
df["INTUBATION"].value_counts()

NO     1350
YES      89
Name: INTUBATION, dtype: int64

In [6]:
df["INTENSIVE CARE"].value_counts()

NO     1270
YES     169
Name: INTENSIVE CARE, dtype: int64

In [7]:
df.head()

,ID,HEIGHT,WEIGHT,INTENSIVE CARE,INTUBATION,WBC 1,NE# 1,LY# 1,MO# 1,EO# 1,...,SEDIM 1,PT 1,APTT 1,INR 1,D-DIMER 1,FIBRINOGEN 1,PROCALCITONIN 1,IgA 1,IgM 1,IgG 1
0,1,164.580078,74.953488,NO,NO,9.08,5.27,2.99,0.55,0.21,...,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0
1,2,174.000000,87.000000,NO,NO,11.31,7.86,2.81,0.48,0.12,...,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0
2,4,182.000000,93.000000,NO,NO,12.42,7.05,4.16,0.90,0.25,...,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0
3,6,164.580078,74.953488,NO,NO,8.19,5.25,2.03,0.53,0.29,...,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0
4,12,164.580078,74.953488,NO,NO,9.56,7.25,0.94,1.08,0.25,...,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0


## INTUBATION

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

In [11]:
xgb=XGBClassifier().fit(X_train, y_train)

[11:37:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [12]:
xgb

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [13]:
y_pred=xgb.predict(X_test)

## Model Tuning

In [14]:
xgb_params={"learing_rate":[0.1,0.01,0.5],
           "max_depth":[2,3,4,4,8],
           "n_estimators":[100,200,500,1000],
           "colsample_bytree":[0.4,0.7,1]}

In [21]:
xgb_cv_model = GridSearchCV(xgb, xgb_params, cv=10, n_jobs=-1, verbose=2).fit(X_train, y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits
[12:04:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "learing_rate" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:04:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [52]:
##xgb_cv_model.best_params_
xgb_model = XGBClassifier(learing_rate=0.1,
                        max_depth=8,
                        n_estimators=500,
                        colsample_bytree=0.7)
xgb_tuned=xgb_model.fit(X_train, y_train)

[14:57:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "learing_rate" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:57:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [53]:
y_pred = xgb_tuned.predict(X_test)
pred=pd.DataFrame({'INTUBATION':y_test, 'INTUBATION_PREDICT':y_pred})

In [54]:
pred

,INTUBATION,INTUBATION_PREDICT
969,NO,NO
179,NO,NO
470,NO,NO
1256,NO,NO
790,NO,NO
...,...,...
701,NO,NO
38,NO,NO
542,NO,NO
704,NO,NO


In [55]:
confusion = metrics.confusion_matrix(pred.INTUBATION, pred.INTUBATION_PREDICT)
confusion

array([[344,   1],
       [ 15,   0]], dtype=int64)

In [56]:
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [57]:
"Senvitiy:" 
TP / float(TP+FN)

0.0

In [58]:
"Specificity:"
TN / float(TN+FP)

0.9971014492753624

In [59]:
cl_report= classification_report(y_test,y_pred)

In [60]:
print(cl_report)

              precision    recall  f1-score   support

          NO       0.96      1.00      0.98       345
         YES       0.00      0.00      0.00        15

    accuracy                           0.96       360
   macro avg       0.48      0.50      0.49       360
weighted avg       0.92      0.96      0.94       360



## INTENSIVE CARE

In [43]:
X_train, X_test, z_train, z_test = train_test_split(X,z,test_size=0.25)

In [44]:
xgb_modelIC =XGBClassifier().fit(X_train, z_train)
xgb_modelIC

[14:45:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

## Model Tuning

In [45]:
xgbIC_params={"learing_rate":[0.1,0.01,0.5],
           "max_depth":[2,3,4,4,8],
           "n_estimators":[100,200,500,1000],
           "colsample_bytree":[0.4,0.7,1]}

In [46]:
xgbIC_cv_model = GridSearchCV(xgb_modelIC, xgbIC_params, cv=10, n_jobs=-1, verbose=2).fit(X_train, z_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits
[14:51:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "learing_rate" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:51:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [63]:
xgbIC_cv_model.best_params_

{'colsample_bytree': 1,
 'learing_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 100}

In [65]:
xgbIC_model = XGBClassifier(learing_rate=0.1,
                        max_depth=3,
                        n_estimators=100,
                        colsample_bytree=1)
xgbIC_tuned=xgbIC_model.fit(X_train, z_train)

[15:00:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "learing_rate" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:00:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [68]:
z_pred = xgbIC_tuned.predict(X_test)
pred=pd.DataFrame({'INTENSIVECARE':z_test, 'INTENSIVECARE_PREDICT':z_pred})

In [69]:
pred

,INTENSIVECARE,INTENSIVECARE_PREDICT
520,NO,NO
304,NO,NO
1272,NO,NO
613,NO,NO
1366,NO,NO
...,...,...
1092,NO,NO
744,NO,NO
1155,YES,YES
369,NO,NO


In [70]:
confusion = metrics.confusion_matrix(pred.INTENSIVECARE, pred.INTENSIVECARE_PREDICT)
confusion

array([[317,   7],
       [ 21,  15]], dtype=int64)

In [71]:
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [72]:
"Senvitiy:" 
TP / float(TP+FN)

0.4166666666666667

In [73]:
"Specificity:"
TN / float(TN+FP)

0.9783950617283951

In [74]:
cl_report= classification_report(y_test,y_pred)

In [75]:
print(cl_report)

              precision    recall  f1-score   support

          NO       0.96      1.00      0.98       345
         YES       0.00      0.00      0.00        15

    accuracy                           0.96       360
   macro avg       0.48      0.50      0.49       360
weighted avg       0.92      0.96      0.94       360

